In [ ]:
import os
import dlib
import glob
import random as rn
import math
import csv
import gc

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

os.environ['TF_DETERMINISTIC_OPS'] = 'true'
rn.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
import keras
from keras import backend as K
print(tf.__version__)
print(keras.__version__)
print(tf.config.experimental.list_physical_devices())

In [ ]:
from keras_video_datagen import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()

train_data = datagen.flow_from_directory('data/train', 
                                         target_size=(70, 140), 
                                         batch_size=5, 
                                         frames_per_step=13, 
                                         shuffle=False, 
                                         color_mode='rgb')

val_data = datagen.flow_from_directory('data/validation', 
                                       target_size=(70, 140), 
                                       batch_size=5, 
                                       frames_per_step=13, 
                                       shuffle=False, 
                                       color_mode='rgb')

In [ ]:
x, y = train_data.next()

print(x.shape)
print(y.shape)

print(y[0])

In [ ]:
print(math.ceil(13559/(5*13)))
print(math.ceil((2366/(5*13))))

In [ ]:
from keras.layers import ZeroPadding3D, Conv3D, BatchNormalization, Activation, MaxPooling3D, TimeDistributed, Flatten, Bidirectional, GRU, Dense

def build_model():
    model_small = keras.models.Sequential()

    model_small.add(ZeroPadding3D(padding=(1, 2, 2), input_shape=(13, 70, 140, 3)))
    model_small.add(Conv3D(32, (3, 5, 5), strides=(1, 2, 2), use_bias=False))
    model_small.add(BatchNormalization(momentum=0.99))
    model_small.add(Activation('relu'))
    model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

    model_small.add(ZeroPadding3D(padding=(1, 2, 2)))
    model_small.add(Conv3D(64, (3, 5, 5), strides=(1, 1, 1), use_bias=False))
    model_small.add(BatchNormalization(momentum=0.99))
    model_small.add(Activation('relu'))
    model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

    model_small.add(ZeroPadding3D(padding=(1, 1, 1)))
    model_small.add(Conv3D(96, (3, 3, 3), strides=(1, 2, 2), use_bias=False))
    model_small.add(BatchNormalization(momentum=0.99))
    model_small.add(Activation('relu'))
    model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

    model_small.add(TimeDistributed(Flatten()))

    model_small.add(Bidirectional(GRU(65, return_sequences=False, activation='tanh')))

    model_small.add(Dense(10, activation='softmax'))

    return model_small

In [ ]:
def set_record():
    record = {}
    record['loss'] = []
    record['accuracy'] = []
    record['val_loss'] = []
    record['val_accuracy'] = []

    record['loss'] += history.history['loss']
    record['accuracy'] += history.history['accuracy']
    record['val_loss'] += history.history['val_loss']
    record['val_accuracy'] += history.history['val_accuracy']

    return record

def play_sound(duration = 0.1, freq = 310):
    for time in range(10):
        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

In [ ]:
lr = 1e-4
momentum = 0.95
index = 2

model = build_model()
sgd = keras.optimizers.SGD(lr=lr, momentum=momentum, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print("\nRun #%d. Learning rate = %f, Momentum = %f" % (index, K.eval(model.optimizer.lr), K.eval(model.optimizer.momentum)))
csv_file = "%d_record_%f_%f.csv" % (index, K.eval(model.optimizer.lr), K.eval(model.optimizer.momentum))

# print(model.layers[6].kernel[0][0][0][0][:4])
history = model.fit_generator(train_data, epochs=100, steps_per_epoch=209, 
                            validation_data=val_data, validation_steps=37, shuffle=False)
# print(model.layers[6].kernel[0][0][0][0][:4])

record = set_record()
with open(csv_file, 'w') as f:
    w = csv.writer(f)
    w.writerow(record.keys())
    for i in range(len(record['loss'])):
        a = []
        for idx in range(len(record)):
            a.append(list(record.values())[idx][i])
        w.writerow(a)
os.remove(csv_file)

del model
del history
del sgd
del record
for i in range(3): gc.collect()

keras.backend.clear_session()
tf.random.set_seed(0)

index += 1

play_sound()